In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
from pymcdm.methods import TOPSIS,COPRAS,PROMETHEE_II,WSM,VIKOR
from pymcdm.helpers import rrankdata
from pymcdm.correlations import weighted_spearman
from pyDecision.algorithm import saw_method

 
prefs = []
ranks = []
ranking_dataset = pd.DataFrame({'Rankings': range(1,50)})
df = pd.read_excel('Set50 Data/KWI_BackTestData.xlsx',sheet_name=['2019-1','2019-2','2019-3','2019-4','2020-1','2020-2','2020-3','2020-4','2021-1','2021-2','2021-3','2021-4','2022-1','2022-2','2022-3','2022-4','2023-1','2023-2','2023-3','2023-4','2024-1','2024-2'])

In [2]:

def four_MCDM(df):
    weight = np.array([0.184,0.137,0.225,0.109,0.131,0.069,0.095,0.032,0.019])
    df1 = df.drop('Company',axis=1)
    use_df = np.array(df1)
    types = np.array([1,-1,-1,1,1,1,1,1,-1])

    methods = [TOPSIS(), COPRAS(), PROMETHEE_II('usual'),VIKOR()] 

    for method in methods:
        pref = method(use_df, weight, types)
        rank = method.rank(pref)
    
        prefs.append(pref)
        ranks.append(rank)


In [3]:
def saw_MCDM(df):
    criterion = ['max','min','min','max','max','max','max','max','min']
    weight = np.array([0.184,0.137,0.225,0.109,0.131,0.069,0.095,0.032,0.019])
    df1 = df.drop('Company',axis=1)
    use_df = np.array(df1)
    rank_saw = saw_method(use_df,criterion,weight,graph=False)
    saw_rank = []
    for i in rank_saw:
        saw_rank.append(i[1])
    prefs.append(saw_rank)
    bc = rrankdata(saw_rank)
    ranks.append(bc)

In [4]:
# method_names = ['TOPSIS','COPRAS','PROMETHEE','VIKOR','SAW']
# stock_name20 = ['AWC','BTS','CPALL','CPN','CRC','DELTA','GPSC','HMPRO','IVL','KBANK','KTB','KTC','LH','MINT','PTT','PTTEP','PTTGC','SCC','SCGP','TISCO']
# a = [f'Stock_{{{i+1}}}' for i in range(len(prefs[0]))]
# pd.DataFrame(zip(*ranks), columns=method_names).astype('int')

In [5]:
def spearman(df2,quarter):
    ranking_corr = np.zeros((5,5))
    spearman_sum_wei = [0,0,0,0,0]
    temp = 0
    for i in range(5):
        for j in range(i+1):
            ranking_corr[i][j] = weighted_spearman(ranks[i],ranks[j]).round(5)
            spearman_sum_wei[j] += ranking_corr[i][j]
            temp += ranking_corr[i][j]

    for i in range(1,5):
        for j in range(i):
            spearman_sum_wei[i] += ranking_corr[i][j]

    spearman_sum_wei /= temp
    final_Score = [0] * 49
    for i in range(5):
        for j in range(49):
            final_Score[j] += spearman_sum_wei[i] * prefs[i][j]
    final_Rank = rrankdata(final_Score)
    quarter_rank = pd.DataFrame(zip(final_Rank),columns=['Ranking'], index=df2['Company']).astype('int').sort_values(['Ranking'])
    quarter_rank = quarter_rank.reset_index()
    quarter_rank = quarter_rank.rename(columns={'Company':quarter})
    return quarter_rank


In [6]:
# spearman_sum_wei /= temp
# spearman_sum_wei
# final_Score = [0] * 49
# for i in range(5):
#     for j in range(49):
#         final_Score[j] += spearman_sum_wei[i] * prefs[i][j]
# final_Rank = rrankdata(final_Score)
# quarter_rank = pd.DataFrame(zip(final_Rank),columns=['Ranking'], index=df['Company']).astype('int').sort_values(['Ranking'])
# pd.concat(ranking_dataset,quarter_rank,axis=1)

In [7]:
df = pd.read_excel('Set50 Data/KWI_BackTestData.xlsx',sheet_name=['2019-1','2019-2','2019-3','2019-4','2020-1','2020-2','2020-3','2020-4','2021-1','2021-2','2021-3','2021-4','2022-1','2022-2','2022-3','2022-4','2023-1','2023-2','2023-3','2023-4','2024-1','2024-2','2024-3'])
sheet_name=['2019-1','2019-2','2019-3','2019-4','2020-1','2020-2','2020-3','2020-4','2021-1','2021-2','2021-3','2021-4','2022-1','2022-2','2022-3','2022-4','2023-1','2023-2','2023-3','2023-4','2024-1','2024-2','2024-3']
for quarter in sheet_name:
    df2 = df[quarter]
    four_MCDM(df2)
    saw_MCDM(df2)
    quarter_rank = spearman(df2,quarter)
    ranking_dataset = pd.concat([ranking_dataset,quarter_rank[quarter]],axis=1)
    ranks = []
    prefs = []
ranking_dataset.to_excel("ranking_results.xlsx", index=False)

c:\Users\patra\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyDecision\algorithm\saw.py:39: RuntimeWarning: invalid value encountered in divide
  X[:,i] = (np.min(X[:,i]) / X[:,i])*weights[i]
c:\Users\patra\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyDecision\algorithm\saw.py:39: RuntimeWarning: invalid value encountered in divide
  X[:,i] = (np.min(X[:,i]) / X[:,i])*weights[i]
c:\Users\patra\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyDecision\algorithm\saw.py:39: RuntimeWarning: divide by zero encountered in divide
  X[:,i] = (np.min(X[:,i]) / X[:,i])*weights[i]
c:\Users\patra\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyDecision\algorithm\saw.py:39: RuntimeWarning: invalid value encountered in divide
  X[:,i] = (np.min(X[:,i]) / X[:,i])*weights[i]
c:\Users\patra\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyDecision\algorithm\saw.py:39: RuntimeWarning: invalid value encountered in divide
  X[:,i] = (np.min(X[:

a1: 0.281
a2: 0.244
a3: 0.181
a4: 0.253
a5: 0.267
a6: 0.264
a7: 0.204
a8: 0.235
a9: 0.175
a10: 0.223
a11: 0.166
a12: 0.204
a13: 0.234
a14: 0.241
a15: 0.196
a16: 0.235
a17: nan
a18: 0.115
a19: 0.279
a20: 0.31
a21: 0.271
a22: 0.222
a23: 0.244
a24: 0.271
a25: 0.327
a26: 0.158
a27: 0.207
a28: 0.239
a29: 0.235
a30: 0.18
a31: 0.265
a32: 0.208
a33: 0.198
a34: 0.287
a35: 0.213
a36: 0.416
a37: 0.211
a38: 0.248
a39: 0.235
a40: 0.242
a41: 0.302
a42: nan
a43: 0.263
a44: 0.315
a45: 0.158
a46: 0.187
a47: 0.135
a48: 0.178
a49: 0.268
a1: 0.331
a2: 0.285
a3: 0.386
a4: 0.496
a5: 0.303
a6: 0.363
a7: 0.272
a8: 0.306
a9: 0.289
a10: 0.355
a11: 0.253
a12: 0.277
a13: 0.291
a14: 0.282
a15: 0.339
a16: 0.284
a17: nan
a18: 0.129
a19: 0.319
a20: 0.479
a21: 0.319
a22: 0.299
a23: 0.341
a24: 0.318
a25: 0.4
a26: 0.329
a27: 0.336
a28: 0.424
a29: 0.422
a30: 0.427
a31: 0.325
a32: 0.322
a33: 0.258
a34: 0.341
a35: 0.359
a36: 0.341
a37: 0.423
a38: 0.386
a39: 0.307
a40: 0.434
a41: 0.39
a42: 0.357
a43: 0.501
a44: 0.485
a45: 0